In [ ]:
# botter本を参考に学習を自前実装
import datetime

import lightgbm as lgb
import numpy as np
import polars as pl
import matplotlib.pyplot as plt
import talib

import stock

In [ ]:
symbol = "BTC_JPY"
interval_minutes = 15
train_ratio = 0.7
pips = 1

# 損切り戦略
max_loss_rate = 0.1
max_hold_timesteps = 10

In [ ]:
fetcher = stock.io.gmo.GMOFethcer()
df = fetcher.fetch_ohlc(symbol, interval=datetime.timedelta(minutes=interval_minutes))

In [ ]:
df = stock.crypto.feature.calc_features(df).filter(
    pl.all_horizontal(pl.col(pl.Float32, pl.Float64).is_not_nan())
)

train_data_num = (int)(len(df) * train_ratio)
train_df = df[:train_data_num]
test_df = df[train_data_num:]

In [ ]:
# botter本に従った逆張り戦略
def calc_force_entry_price(entry_price: np.ndarray, low: np.ndarray, pips: float):
    fep = np.zeros(entry_price.shape)
    fet = np.zeros(entry_price.shape)
    fep[:] = np.nan

    is_executable = np.round(low[1:] / pips) < np.round(entry_price[:-1] / pips)

    start_i = 0
    for i in range(len(is_executable)):
        fet[start_i:i + 1] += 1
        if is_executable[i]:
            fep[start_i:i + 1] = entry_price[i]
            start_i = i + 1
    return fep, fet


def calc_return(
    df, max_loss_rate=max_loss_rate, max_hold_timesteps=max_hold_timesteps
):
    df = df.with_columns(
        (((pl.col("close") - pl.col("ATR") *0.5) / pips).round()  * pips).alias("buy_point"),
        (((pl.col("close") + pl.col("ATR") * 0.5) / pips).round() * pips).alias("sell_point"),
    ).with_columns(
        (pl.col("buy_point") > pl.col("low").shift(-1)).alias("buy_executed"),    
        (pl.col("sell_point") < pl.col("high").shift(-1)).alias("sell_executed"),
    )
    
    buy_fep, buy_fet = calc_force_entry_price(df["buy_point"].to_numpy(), df["low"].to_numpy(), pips=1.0)
    sell_fep, sell_fet = calc_force_entry_price(-df["sell_point"].to_numpy(), -df["high"].to_numpy(), pips=1.0)
    sell_fep *= -1.0

    def calc_profits(start, low, close, is_executed, buy, sell, sell_time, max_loss_rate):
        profits = np.zeros(len(df))
        for i in range(len(df)):
            if not is_executed[i]:
                continue
            sell_price = sell[i + 1]
            duration = int(min(sell_time[i + 1], max_hold_timesteps))
            if sell_time[i + 1] > max_hold_timesteps:
                sell_price = close[i + 1 + max_hold_timesteps]

            loss_cut_price = np.round(buy[i] * (1.0 - max_loss_rate) / pips) * pips
            for j in range(duration):
                if low[i + 1 + j] < loss_cut_price:
                    sell_price = min(start[i + 1 + j], loss_cut_price)
                    break
            profits[i] = sell_price / buy[i] - 1.0
        return profits
    
    buy_profit = calc_profits(
        df["open"].to_numpy(), df["low"].to_numpy(), df["close"].to_numpy(), 
        is_executed=df["buy_executed"].to_numpy(), 
        buy=buy_fep,
        sell=sell_fep,
        sell_time=sell_fet,
        max_loss_rate=max_loss_rate
    )

    sell_profit = 1.0 / (calc_profits(
        -df["open"].to_numpy(), -df["high"].to_numpy(), -df["close"].to_numpy(), 
        is_executed=df["sell_executed"].to_numpy(), 
        buy=-sell_fep,
        sell=-buy_fep,
        sell_time=buy_fet,
        max_loss_rate=-max_loss_rate
    ) + 1.0) - 1.0
    return buy_profit, sell_profit

In [ ]:
buy_profit, sell_profit = calc_return(df)
plt.plot(buy_profit.cumsum())
plt.plot(sell_profit.cumsum())
plt.plot(df["close"] / df["close"][-1] * 10)

In [ ]:
train_features = [
    'ADX',
    'ADXR',
    'APO',
    'AROON_aroondown',
    'AROON_aroonup',
    'AROONOSC',
    'CCI',
    'DX',
    'MACD_macd',
    'MACD_macdsignal',
    'MACD_macdhist',
    'MFI',
#     'MINUS_DI',
#     'MINUS_DM',
    'MOM',
#     'PLUS_DI',
#     'PLUS_DM',
    'RSI',
    'STOCH_slowk',
    'STOCH_slowd',
    'STOCHF_fastk',
#     'STOCHRSI_fastd',
    'ULTOSC',
    'WILLR',
#     'ADOSC',
#     'NATR',
    'HT_DCPERIOD',
    'HT_DCPHASE',
    'HT_PHASOR_inphase',
    'HT_PHASOR_quadrature',
    'HT_TRENDMODE',
    'BETA',
    'LINEARREG',
    'LINEARREG_ANGLE',
    'LINEARREG_INTERCEPT',
    'LINEARREG_SLOPE',
    'STDDEV',
    'BBANDS_upperband',
    'BBANDS_middleband',
    'BBANDS_lowerband',
    'DEMA',
    'EMA',
    'HT_TRENDLINE',
    'KAMA',
    'MA',
    'MIDPOINT',
    'T3',
    'TEMA',
    'TRIMA',
    'WMA',
]

In [ ]:
train_x = df.select(*train_features).to_numpy()
estimator = lgb.LGBMRegressor(n_jobs=1, random_state=1)
estimator.fit(train_x, buy_profit)

In [ ]:
test_df = stock.crypto.feature.calc_features(test_df).filter()
test_buy_profit, test_sell_profit = calc_return(test_df)

In [ ]:
test_pred = estimator.predict(test_df.select(*train_features).to_numpy())